In [ ]:
!pip install langchain
!pip install PyPDF2
!pip install qdrant_client
!pip install langchain_openai
!pip install pypdf
!pip install openai

In [2]:
import os

from langchain.document_loaders import PyPDFLoader
from langchain.text_splitter import CharacterTextSplitter
from langchain.embeddings.openai import OpenAIEmbeddings
from langchain.vectorstores import Qdrant
from langchain.chains import ConversationalRetrievalChain
from langchain.document_loaders import TextLoader
from qdrant_client import QdrantClient,models
from qdrant_client.http.models import PointStruct
from openai import OpenAI

In [3]:
os.environ["OPENAI_API_KEY"]=""
os.environ["QDRANT_URL"]=""

In [ ]:
#create new cluseter in qdrant
record=0

connection = QdrantClient(
    url=os.environ.get("QDRANT_URL"),
    api_key="",
)


connection.recreate_collection(
    collection_name="health_report",
    vectors_config=models.VectorParams(size=1536, distance=models.Distance.COSINE),
)
print("Create collection reponse:", connection)

info = connection.get_collection(collection_name="health_report")

print("Collection info:", info)
for get_info in info:
  print(get_info)

In [6]:
def qdrants_load(collection_name):
    return Qdrant(
        client=connection,
        collection_name=collection_name,
        embeddings=get_embeddings()
    )

In [7]:
def read_files_in_pdf_dir():
    docs = []
    for filename in os.listdir("./pdf/"):
        filepath = os.path.join("./pdf/", filename)
        if filename.endswith('.txt'):
            loader = TextLoader(filepath)
            text_splitter = CharacterTextSplitter(chunk_size=300, chunk_overlap=50)
            pages = loader.load_and_split(text_splitter)
            docs.extend(pages)
        elif filename.endswith('.pdf'):
            loader = PyPDFLoader(filepath)
            text_splitter = CharacterTextSplitter(chunk_size=300, chunk_overlap=50)
            pages = loader.load_and_split(text_splitter)
            docs.extend(pages)
    if len(docs) != 0:
        print(docs)
        Qdrant = qdrants_load("health_report")
        Qdrant.add_documents(docs)
        print("succeded to read files!")


In [8]:
def get_embeddings():
  return OpenAIEmbeddings(
    openai_api_key=os.environ.get("OPENAI_API_KEY"),
    chunk_size=1
  )

In [9]:
from langchain_openai import ChatOpenAI
def get_chat_model():
  return ChatOpenAI(model_name="gpt-3.5-turbo-0125", temperature=0)

In [10]:
def get_document_store(docs, embeddings):
  upsert = Qdrant.from_documents(
    docs,
    embeddings,
    url=os.environ.get("QDRANT_URL"),
    collection_name="health_report",
    api_key="PtYX2su0b_Xof19YN54ybCUvIZgdA94HqDe0vPUHBQ8CNu7Moun0VQ"
  )
  print(upsert)
  return upsert

In [11]:
def ask_question_with_context(qa, question, chat_history):

    query = ""
    result = qa({"question": question, "chat_history": chat_history})
    print("answer:", result["answer"])
    chat_history = [(query, result["answer"])]
    return chat_history

In [ ]:
def main():

    read_files_in_pdf_dir()
    llm = get_chat_model()

if __name__ == "__main__":
    main()
